# Ficha 6

Na Figura 1 é apresentado o esquema lógico, nomeadamente modelo relacional, da base de dados de uma
determinada clínica.
A base de dados que o esquema representa está integrada no sistema operacional principal da clínica e su-
porta todos os processos relacionados com o registo e a faturação das consultas realizadas. É de referir que o
valor pago pelo paciente em cada consulta pode ser superior ao valor de referência para a especialialidade do
médico (preço), uma vez que podem ser pagos procedimentos efetuados durante a realização da consulta.
Na folha de exercícios anterior utilizou o MySQL Workbench com o ficheiro “Ficha5_ModeloLogico.mwb”,
disponibilizado juntamente com a 5.a ficha prática laboratorial, de modo a gerar o respetivo esquema físico para
a base de dados em questão.
Com base no esquema físico gerado, pretende-se que:
1. Povoe a base de dados criada, utilizando o ficheiro “Ficha6_PovoarTabelas.sql” disponibilizado juntamente
com esta ficha prática laboratorial. O ficheiro disponibiliza igualmente a função “idade” que será muito
útil para responder a alguns dos pontos da questão 2.

![done](tabelas-povoadas.png)

2. Utilizando SQL, desenvolva os comandos necessários para responder às seguintes questões:

Foi criada uma função `anservico` para facilitar o cálculo dos anos de serviço.

``` sql

CREATE DEFINER=`root`@`localhost` FUNCTION `anservico`(dta date) RETURNS int(3)
BEGIN
RETURN TIMESTAMPDIFF(YEAR, dta, CURDATE());
END

```

* a) Qual é o nome dos médicos com mais de 10 anos de serviço?

``` sql
SELECT * FROM medico WHERE anservico(data_inicio_servico) > 10
```
|Resultado|
|-|
|123456	Jose Maria	Rua de Cima n.º 223, 5.º DTO	4700-001	1970-01-23	1	1990-02-23|
|223456	Antonio Pinto	Rua de Baixo n.º 125, 2.º ESQ	4700-003	1991-11-21	2	1990-01-23|
|323456	Manuel Maria Neves	Rua de Sul n.º 789, 9.º DTO	4700-002	1964-03-24	2	1979-07-23|
|423456	Pedro Pinto Silva	Rua de Este n.º 456, 1.º DTO	4750-002	1978-02-13	3	1980-01-23|
|523456	Marta Catarina	Rua de Cima n.º 120, 3.º ESQ	4700-001	1960-09-11	1	1971-05-23|



* b) Qual é o nome de todos os médicos e a respetiva especialidade que cada um exerce?

``` sql
SELECT medico.nome, especialidade.designacao 
FROM medico
JOIN especialidade ON medico.especialidade = especialidade.id_especialidade
```

|Resultado|
|-|
|Jose Maria	Oftalmologia|
|Antonio Pinto	Clinica Geral|
|Manuel Maria Neves	Clinica Geral|
|Pedro Pinto Silva	Cardiologia|
|Marta Catarina	Oftalmologia|

* c) Qual é o nome e a morada dos pacientes residentes na localidade de Braga?

``` sql
SELECT nome, morada
FROM paciente
JOIN codigo_postal ON paciente.codigo_postal = codigo_postal.codigo_postal
WHERE codigo_postal.localidade = 'Braga'
```
|Resultado|
|-|


* d) Qual é o nome dos médicos da especialidade de Oftalmologia?

``` sql
SELECT medico.nome, especialidade.designacao 
FROM medico
JOIN especialidade ON medico.especialidade = especialidade.id_especialidade
WHERE especialidade.designacao = 'Oftalmologia'
```
|Resultado|
|-|
|Jose Maria	Oftalmologia|
|Marta Catarina	Oftalmologia|


* e) Qual é o nome e a idade dos médicos com mais de 40 anos de idade da especialidade de Clínica Geral?

``` sql
SELECT nome, idade(data_nascimento) 
FROM medico
JOIN especialidade ON medico.especialidade = especialidade.id_especialidade
WHERE especialidade.designacao = 'Clínica Geral'

```
|Resultado|
|-|
Antonio Pinto	32
Manuel Maria Neves	59

* f) Qual é o nome dos médicos da especialidade de Oftalmologia que consultaram pacientes da localidade de Braga?

``` sql
SELECT DISTINCT medico.nome AS nome_medico, paciente.nome AS nome_paciente, especialidade.designacao as especialidade
FROM medico
JOIN especialidade ON medico.especialidade = especialidade.id_especialidade
JOIN consulta ON medico.id_medico = consulta.id_medico
JOIN paciente ON consulta.id_paciente = paciente.id_paciente
JOIN codigo_postal ON paciente.codigo_postal = codigo_postal.codigo_postal
WHERE especialidade.designacao = 'Oftalmologia' AND codigo_postal.localidade = 'Braga';

```

|Resultado|
|-|
|Jose Maria	Manuel Marques	Oftalmologia
Jose Maria	Manuel Marques	Oftalmologia
Jose Maria	Jose Manuel Marques	Oftalmologia

* g) Qual é o nome e os anos de serviço dos médicos com mais de 50 anos de idade que deram consultas a partir das 12h a pacientes com menos de 20 anos de idade?

``` sql
SELECT medico.nome AS nome_medico, anservico(medico.data_inicio_servico) AS anos_servico, idade(medico.data_nascimento) AS idade_medico, paciente.nome AS nome_paciente, idade(paciente.data_nascimento) AS idade_paciente
FROM medico
JOIN consulta ON consulta.id_medico = medico.id_medico
JOIN paciente ON consulta.id_paciente = paciente.id_paciente
WHERE idade(medico.data_nascimento) > 50 AND idade(paciente.data_nascimento) < 20

```
|Resultado|
|-|
|Manuel Maria Neves	44	59	Maria Jose Silva	11|

* h) Qual é o nome e a idade dos pacientes com mais de 10 anos de idade que nunca foram consultados na especialidade de Oftalmologia?

``` sql
SELECT paciente.nome AS paciente, idade(paciente.data_nascimento) AS idade, especialidade.designacao AS especialidade_consultada
FROM medico
JOIN consulta ON consulta.id_medico = medico.id_medico
JOIN paciente ON consulta.id_paciente = paciente.id_paciente
JOIN especialidade ON especialidade.id_especialidade = medico.especialidade
WHERE especialidade.designacao != 'Oftalmologia' AND idade(paciente.data_nascimento) > 10
```
Manuel Marques	53	Clinica Geral
Pedro Paulo	32	Clinica Geral
Pedro Paulo	32	Clinica Geral
Pedro Paulo	32	Clinica Geral
Pedro Paulo	32	Clinica Geral
Jacinto Homem	41	Clinica Geral
Sara Catarina Costa	37	Clinica Geral
Jose Carlos Moreira	53	Clinica Geral
Joana Silva	13	Clinica Geral
Maria Jose Silva	11	Clinica Geral
Sara Catarina Costa	37	Clinica Geral
Rosa Maria Carvalho	22	Clinica Geral
Manuel Marques	53	Cardiologia
Maria Jose Silva	11	Cardiologia
Sara Catarina Costa	37	Cardiologia
Rosa Maria Carvalho	22	Cardiologia

* i) Qual é o nome das especialidades que tiveram consultas no mês de janeiro de 2016?

``` sql
SELECT DISTINCT especialidade.designacao as especialidade_jan_16
FROM especialidade
JOIN medico ON especialidade.id_especialidade = medico.especialidade
JOIN consulta ON consulta.id_medico = medico.id_medico
WHERE year(consulta.data_hora) = 2016 AND month(consulta.data_hora) = 1
```
|Resultado|
|-|
|Oftalmologia|
|Clinica Geral|

* j) Qual é o nome dos médicos com mais de 30 anos de idade ou menos de 5 anos de serviço?

``` sql
SELECT nome FROM medico
WHERE idade(data_nascimento) > 30 AND anservico(data_inicio_servico) >5
```

Jose Maria
Antonio Pinto
Manuel Maria Neves
Pedro Pinto Silva
Marta Catarina


* k) Qual é o nome e a idade dos médicos de Clínica Geral que não consultaram em janeiro de 2016?

``` sql
SELECT DISTINCT nome, idade(data_nascimento) AS idade FROM medico
JOIN especialidade ON especialidade.id_especialidade = medico.especialidade
JOIN consulta ON consulta.id_medico = medico.id_medico
WHERE especialidade.designacao = 'Clínica Geral' AND (month(consulta.data_hora) IS NULL OR (month(consulta.data_hora) != 1 AND year(consulta.data_hora) != 2016))

```

* l) Qual é o nome e a idade dos pacientes que já foram consultados por todos os médicos?

``` sql
SELECT paciente.nome, idade(paciente.data_nascimento) AS idade FROM paciente
JOIN consulta ON consulta.id_paciente = paciente.id_paciente
JOIN medico ON consulta.id_medico = medico.id_medico
GROUP BY paciente.nome, idade
HAVING count(*) = (SELECT COUNT(*) FROM medico)
```
Sara Catarina Costa	37

* m) Qual é o nome das especialidades que tiveram consultas no mês de janeiro e março de 2016?

``` sql
SELECT DISTINCT especialidade.designacao as especialidade_jan_e_mar_16
FROM especialidade
JOIN medico ON especialidade.id_especialidade = medico.especialidade
JOIN consulta ON consulta.id_medico = medico.id_medico
WHERE year(consulta.data_hora) = 2016 AND (month(consulta.data_hora) = 1 OR month(consulta.data_hora) = 3)

```
|Resultado|
|-|
|Oftalmologia|
|Clinica Geral|

* n) Qual é o nome dos médicos que nunca consultaram pacientes residentes em Braga?

``` sql
SELECT DISTINCT medico.nome FROM medico
JOIN consulta ON consulta.id_medico = medico.id_medico
JOIN paciente ON consulta.id_paciente = paciente.id_paciente
JOIN codigo_postal ON codigo_postal.codigo_postal = paciente.codigo_postal
WHERE codigo_postal.localidade != 'Braga'
```

* o) Qual é o nome e a idade dos pacientes que só foram consultados a Clínica Geral?

``` sql
SELECT DISTINCT paciente.nome, idade(paciente.data_nascimento) AS idade FROM paciente
JOIN consulta ON consulta.id_paciente = paciente.id_paciente
JOIN medico ON consulta.id_medico = medico.id_medico
JOIN especialidade ON especialidade.id_especialidade = medico.especialidade
WHERE especialidade.designacao = 'Clínica Geral'

```

